In [64]:
#Import libraries
import pandas as pd

In [65]:
#!pip install TextBlob

In [388]:
dirPath = r"\data"
reviewPath = r"reviews_Tools_and_Home_Improvement_5.json.gz"
#reviewPath = r"reviews_Musical_Instruments_5.json.gz"
reviewPath = r"cleanData_reviews_Musical_Instruments_5.json.gz"
path = dirPath + "\\" + reviewPath
path = "preprocessed_reviews_Musical_Instruments_5.pkl.gz"
path = "preprocessed_reviews_Tools_and_Home_Improvement_5.pkl.gz"

In [389]:
data = pd.read_pickle(path)
data.shape

(123707, 7)

In [390]:
negativeReviews = data.loc[data['trueRating'] == 0, :]
negativeReviews.shape[0]


10105

In [391]:
#data = data.iloc[0:size*5, :]

In [392]:
negativeReviews.tail(2)

,masterIndex,asin,reviewText,helpful,overall,trueRating,CleanedJoin
123477,134239,B00JPBDL9W,I have converted three-quarters of my home to ...,"[0, 0]",2.0,0.0,i have convert three quarter of my home to lea...
123556,134322,B00JXSASK2,Amazing that they designed this thing without ...,"[2, 2]",1.0,0.0,amaze that they design this thing without real...


In [393]:
#Pad more negative reviews for better training purposes
testSample = pd.concat([data, negativeReviews])
#testSample = data
#testSample.reset_index(drop=True, inplace=True)
testSample.tail(1)

,masterIndex,asin,reviewText,helpful,overall,trueRating,CleanedJoin
123556,134322,B00JXSASK2,Amazing that they designed this thing without ...,"[2, 2]",1.0,0.0,amaze that they design this thing without real...


In [394]:
import itertools

largeText = testSample.CleanedJoin.to_list()
#largeText = reviewTextProcessing.reviewText.to_list() # Not too good accuracy around 116 missed with %86.94

len(largeText)

133812

In [395]:
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

In [396]:
num_words = 15000

In [397]:
tokenizer = Tokenizer(num_words=num_words)
tokenizer.fit_on_texts(largeText)
#tokenizer.word_index

In [398]:
X = largeText
y = testSample['trueRating']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.25, random_state=310)

x_train_text = X_train
x_test_text = X_test

x_train_tokens = tokenizer.texts_to_sequences(x_train_text)
x_test_tokens = tokenizer.texts_to_sequences(x_test_text)

In [399]:
len(y_train), len(y_test), len(x_train_text), len(x_test_text)

(100359, 33453, 100359, 33453)

In [400]:
#x_train_tokens[1], x_train_text[1]

In [401]:
import numpy as np

x_train_text[1]
np.array(x_train_tokens[1])
num_tokens = [len(tokens) for tokens in x_train_tokens + x_test_tokens]
num_tokens = np.array(num_tokens)


In [402]:
np.mean(num_tokens), np.max(num_tokens)


(111.61465339431442, 4959)

In [403]:
np.std(num_tokens)

133.85901020345165

In [404]:
max_tokens = np.mean(num_tokens) + 2 * np.std(num_tokens)
max_tokens = int(max_tokens) 
max_tokens


379

In [405]:
np.sum(num_tokens < max_tokens) / len(num_tokens)


0.9588078797118346

In [406]:
pad = 'pre'
#pad = 'post'
x_train_pad = pad_sequences(x_train_tokens, maxlen=max_tokens, padding=pad, truncating=pad)
x_test_pad = pad_sequences(x_test_tokens, maxlen=max_tokens, padding=pad, truncating=pad)
x_train_pad.shape, x_test_pad.shape


((100359, 379), (33453, 379))

In [407]:
for i in range(3, 4):
    print(x_train_pad[i])


[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0 

In [279]:
def tokens_to_string(tokens):
    # Map from tokens back to words.
    words = [inverse_map[token] for token in tokens if token != 0]
    
    # Concatenate all words.
    text = " ".join(words)

    return text

In [280]:
idx = tokenizer.word_index
inverse_map = dict(zip(idx.values(), idx.keys()))

In [281]:
x_train_text[1]

'work awesome'

In [282]:
tokens_to_string(x_train_tokens[1])

'work awesome'

In [284]:
import tensorflow as tf
tf.__version__

'1.13.2'

In [325]:
#abstract CNN
import modelCNNImplementation as modelCNN
import importlib
importlib.reload(modelCNN)

<module 'modelCNNImplementation' from 'C:\\Study\\Projects\\NLP\\modelCNNImplementation.py'>

In [408]:
model_cnn = modelCNN.compileCNNModel(max_tokens=max_tokens)
model_cnn.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 379, 200)          3000000   
_________________________________________________________________
conv1d_5 (Conv1D)            (None, 378, 100)          40100     
_________________________________________________________________
global_max_pooling1d_5 (Glob (None, 100)               0         
_________________________________________________________________
dense_11 (Dense)             (None, 256)               25856     
_________________________________________________________________
dense_12 (Dense)             (None, 1)                 257       
Total params: 3,066,213
Trainable params: 3,066,213
Non-trainable params: 0
_________________________________________________________________


In [409]:
import modelRNNImplementation as modelRNN
import importlib
importlib.reload(modelRNN)

<module 'modelRNNImplementation' from 'C:\\Study\\Projects\\NLP\\modelRNNImplementation.py'>

In [410]:
model_rnn = modelRNN.compileRNNModel(max_tokens=max_tokens)
model_rnn.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
layer_embedding (Embedding)  (None, 379, 24)           360000    
_________________________________________________________________
gru_7 (GRU)                  (None, 379, 16)           1968      
_________________________________________________________________
gru_8 (GRU)                  (None, 379, 8)            600       
_________________________________________________________________
gru_9 (GRU)                  (None, 4)                 156       
_________________________________________________________________
dense_13 (Dense)             (None, 1)                 5         
Total params: 362,729
Trainable params: 362,729
Non-trainable params: 0
_________________________________________________________________


In [411]:
index = len(x_train_pad)
index
len(y_train), len(y_test), len(y_train) + len(y_test)

(100359, 33453, 133812)

In [ ]:
%%time
model_cnn.fit(x_train_pad, y_train, validation_split=0.05, epochs=3, batch_size=36)
#model.fit(x_train_pad, y_train,validation_split=0.05, epochs=3, batch_size=74)

Train on 95341 samples, validate on 5018 samples
Epoch 1/3
26064/95341 [=======>......................] - ETA: 7:07 - loss: 0.2813 - acc: 0.8878

In [ ]:
%%time
model_rnn.fit(x_train_pad, y_train, validation_split=0.05, epochs=3, batch_size=36)
#model.fit(x_train_pad, y_train,validation_split=0.05, epochs=3, batch_size=74)

In [ ]:
#%%time
result = model_cnn.evaluate(x_test_pad, y_test)

In [ ]:
#%%time
rnn_result = model_rnn.evaluate(x_test_pad, y_test)

In [ ]:
print("Accuracy: {0:.2%}".format(rnn_result[1]))

In [ ]:
print("Accuracy: {0:.2%}".format(result[1]))


In [ ]:
#%%time
y_pred = model_cnn.predict(x=x_test_pad)
#y_pred = model_rnn.predict(x=x_test_pad)
y_pred = y_pred.T[0]

In [ ]:
pd.DataFrame(y_pred).describe()

In [ ]:
y_pred.shape

In [ ]:
cls_pred = np.array([1.0 if p>0.5 else 0.0 for p in y_pred])


In [ ]:
len(cls_pred),

In [ ]:
accuracy = (cls_pred==y_test).mean()
accuracy

In [ ]:
sample = len(cls_pred)
sample

In [ ]:
from sklearn.metrics import confusion_matrix

In [ ]:
#Duplicate has been 
tn, fp, fn, tp = confusion_matrix(y_test, cls_pred).ravel()
tn, fp, fn, tp

In [ ]:
len(np.where(y_test==0)[0]),len(np.where(y_test==1)[0])

In [ ]:
len(np.where(y_train==0)[0]),len(np.where(y_train==1)[0])

In [ ]:
cls_true = np.array(y_test)

In [ ]:
y_test.index
y_test.values
result = pd.DataFrame()
result['Index'] = y_test.index
result['TrueRating'] = y_test.values
result['RNNRating'] = cls_pred

In [ ]:
result.shape

In [ ]:
# need to drop the duplicates due to negative padding
result.drop_duplicates(subset="Index", inplace=True)
result.shape

In [ ]:
df = testSample.loc[result.Index.to_list(), :]
df = df.reset_index()
#A.drop_duplicates()
df= df.drop_duplicates(subset="index")
df.rename(columns={"index": "Index"}, inplace=True)
df.shape, df.columns

In [ ]:
df.head(4)

In [ ]:
modelResult = pd.merge(df, result, on="Index", how="inner")
modelResult.head(3)

In [ ]:
import TextBlobImplementation as TextBlob
import importlib
importlib.reload(TextBlob)

In [ ]:
testSample = modelResult.iloc[3, :]['CleanedJoin']

TextBlob.getPolarityFromText(testSample.split(','))

In [ ]:
modelResult['textBlobRating'] = modelResult.CleanedJoin.str.split(',')
modelResult['textBlobRating'] = modelResult.textBlobRating.apply(TextBlob.getPolarityFromText)

In [ ]:
modelResult.head(3)

In [322]:
from sklearn.metrics import confusion_matrix

y_true = modelResult.TrueRating.to_list()
cnn_pred = modelResult.CNNRating.to_list()
textBlob_pred = modelResult.textBlobRating.to_list()

In [385]:
from sklearn.metrics import confusion_matrix

y_true = modelResult.TrueRating.to_list()
rnn_pred = modelResult.RNNRating.to_list()
textBlob_pred = modelResult.textBlobRating.to_list()

In [323]:
tn, fp, fn, tp = confusion_matrix(y_true, cnn_pred).ravel()
tn, fp, fn, tp

(31, 78, 37, 2227)

In [386]:
tn, fp, fn, tp = confusion_matrix(y_true, rnn_pred).ravel()
tn, fp, fn, tp

(113, 88, 34, 2224)

In [387]:
tn, fp, fn, tp = confusion_matrix(y_true, textBlob_pred).ravel()
tn, fp, fn, tp

(50, 151, 112, 2146)

In [312]:
pd.options.display.max_colwidth = 50

#productsReviewData.cleanedText.head(2)

## 10K review comparison

## padding extra negative reviews

##### we pick up two unique negative extras with the sampling when we padded

tn, fp, fn, tp

(174, 27, 44, 2214) CNN for negative reviews padding 
(113, 88, 34, 2224)  RNN with negative review padding
(50, 151, 112, 2146) textBlob with negative padding 

## no padding extra negative reviews

tn, fp, fn, tp

(31, 78, 37, 2227)  CNN without negative reviews padding
(3, 106, 2, 2262)   RNN without negative reviews padding
(27, 82, 106, 2158) textBlob without negative reviews padding


## Multinomial Naive Bayes no padding extra negative reviews

tn, fp, fn, tp

(12, 97, 232, 2032) Multinomial Naive Bayes # n-gram (1,2), num_words = 10000
(26, 83, 393, 1871) Multinomial Naive Bayes # n-gram (1,1), num_words = 3000


